In [30]:
from nltk.tokenize import sent_tokenize, word_tokenize,RegexpTokenizer
from nltk.corpus import wordnet as wn
import csv
import pandas as pd
import csv
import nltk
import numpy as np
import theano as theano
import theano.tensor as T
from utils import *
import operator
from datetime import *
import sys

#test=1
f=open("Sentence.txt","r")
x= f.read()
print()
print("Trained Paragraph:- " )
print(x)
print()
row_of_sentences=sent_tokenize(x);

#tokenizer = RegexpTokenizer(r'\w+')
#tok=tokenizer.tokenize(x)
#tok.insert(0,"SENTENCE_START")
#tok.append("SENTENCE_END")

for i in range (len(row_of_sentences)):
    tok=word_tokenize(row_of_sentences[i])  #list type
    tok.insert(0,"sent_start")
    tok.append("sent_end")
    row_of_sentences[i]=tok;
    #print(tok)
    
df=pd.read_csv("final.csv",names=['Words'])  
df=df['Words']
#df.loc[0]='sent_start'
voc=df.values.tolist()


#vecx=[]
#vecy=[]
index_to_word=dict()
word_to_index=dict()
xtrainlist=[]
ytrainlist=[]

for i in row_of_sentences:
    vecx=[]
    for j in i[:-1]:
        #print (j)
        if(j in voc):
            x=voc.index(j)
            vecx.append(x)
            index_to_word[x]=j
            word_to_index[j]=x
        else:
            x=len(voc)
            df.loc[len]=j
            df.to_csv("final.csv")
            voc=df.values.tolist()
            vecx.append(x)
            index_to_word[x]=j
            word_to_index[j]=x
    xtrainlist.append(vecx)


for i in range(len(row_of_sentences)):
    vecy=[]
    for j in range(1,len(xtrainlist[i])):
        vecy.append(xtrainlist[i][j])
    x=voc.index("sent_end")
    vecy.append(x)    
    ytrainlist.append(vecy)
    
x=voc.index("sent_end")
index_to_word[x]="sent_end"
word_to_index["sent_end"]=x 


X_train=np.asarray(xtrainlist)
Y_train=np.asarray(ytrainlist)

print("Our X_train 2-D Array:-")
print(X_train)
print()

print("Our Y_train 2-D Array:-")
print(Y_train)
print()



Trained Paragraph:- 
Two households, both alike in dignity
(In fair Verona, where we lay our scene),
From ancient grudge break to new mutiny,
Where civil blood makes civil hands unclean.
5From forth the fatal loins of these two foes
A pair of star-crossed lovers take their life,
Whose misadventured piteous overthrows
Doth with their death bury their parents' strife.
The fearful passage of their death-marked love
10And the continuance of their parents' rage,
Which, but their children’s end, naught could remove,
Is now the two hours' traffic of our stage—
The which, if you with patient ears attend,
What here shall miss, our toil shall strive to mend.

Our X_train 2-D Array:-
[list([0, 122, 123, 34, 124, 125, 76, 126, 68, 127, 128, 129, 34, 130, 65, 131, 132, 133, 71, 34, 134, 135, 136, 137, 31, 138, 139, 34, 140, 141, 142, 143, 141, 144, 145, 26])
 list([0, 146, 147, 12, 148, 149, 19, 150, 151, 152, 27, 153, 19, 154, 155, 156, 157, 158, 34, 159, 160, 161, 162, 163, 164, 157, 165, 166, 1

In [50]:
class RNNNumpy:

    
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        self.h=np.random.uniform(0,1,hidden_dim)
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))
    
    def softmax(self,x):
        xt = np.exp(x - np.max(x))
        return xt / np.sum(xt)

    def forward_propagation(self, x):
        # The total number of time steps
        T = len(x)
        # During forward propagation we save all hidden states in s because need them later.
        # We add one additional element for the initial hidden, which we set to 0
        s = np.zeros((T + 1, self.hidden_dim))
        s[-1] = np.zeros(self.hidden_dim)
        # The outputs at each time step. Again, we save them for later.
        o = np.zeros((T, self.word_dim))
        # For each time step...
        for t in np.arange(T):
            # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
            s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
            o[t] = self.softmax(self.V.dot(s[t]))
        return [o, s]

    def calculate_total_loss(self, x, y):
        L = 0
        # For each sentence...
        for i in np.arange(len(y)):
            o, s = self.forward_propagation(x[i])
            # We only care about our prediction of the "correct" words
            correct_word_predictions = o[np.arange(len(y[i])), y[i]]
            # Add to the loss based on how off we were
            L += -1 * np.sum(np.log(correct_word_predictions))
        return L
 
    def calculate_loss(self, x, y):
        # Divide the total loss by the number of training examples
        N = np.sum((len(y_i) for y_i in y))
        return self.calculate_total_loss(x,y)/N

    def bptt(self, x, y):
        T = len(y)
        # Perform forward propagation
        o, s = self.forward_propagation(x)
        # We accumulate the gradients in these variables
        #with open('U.txt') as file:
         #    dLdU= [[float(digit) for digit in line.split()] for line in file]
        #with open('V.txt') as file:
         #    dLdV= [[float(digit) for digit in line.split()] for line in file]
        #with open('W.txt') as file:
         #    dLdW= [[float(digit) for digit in line.split()] for line in file]

        #dLdU=np.asarray(dLdU)
        #dLdV=np.asarray(dLdV)
        #dLdW=np.asarray(dLdW)
        dLdU = np.zeros(self.U.shape)
        dLdV = np.zeros(self.V.shape)
        dLdW = np.zeros(self.W.shape)

        
        delta_o = o
        #print("heya")
        #print(delta_o)
        delta_o[np.arange(len(y)), y] -= 1.
        # For each output backwards...
        for t in np.arange(T)[::-1]:
            #print(delta_o[t].shape)
            #print(s[t].shape)
            #print(s[t])
            dLdV += np.outer(delta_o[t], s[t].T)
            # Initial delta calculation
            delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
            # Backpropagation through time (for at most self.bptt_truncate steps)
            for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
                # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
                dLdW += np.outer(delta_t, s[bptt_step-1])              
                dLdU[:,x[bptt_step]] += delta_t
                # Update delta for next step
                delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
        #a = np.asarray(dLdU)
        #np.savetxt("U.txt", a, delimiter=" ")
        #a = np.asarray(dLdV)
        #np.savetxt("V.txt", a, delimiter=" ")
        #a = np.asarray(dLdW)
        #np.savetxt("W.txt", a, delimiter=" ")
        return [dLdU, dLdV, dLdW]



    # Performs one step of SGD.
    def sgd_step(self, x, y, learning_rate):
        # Calculate the gradients
        dLdU, dLdV, dLdW = self.bptt(x, y)
        # Change parameters according to gradients and learning rate
        self.U -= learning_rate * dLdU
        self.V -= learning_rate * dLdV
        self.W -= learning_rate * dLdW
 

    # Outer SGD Loop
    # - model: The RNN model instance
    # - X_train: The training data set
    # - y_train: The training data labels
    # - learning_rate: Initial learning rate for SGD
    # - nepoch: Number of times to iterate through the complete dataset
    # - evaluate_loss_after: Evaluate the loss after this many epochs
    
    def train_with_sgd(self, X_train, y_train, learning_rate=0.005, nepoch=200, evaluate_loss_after=5):
        # We keep track of the losses so we can plot them later
        losses = []
        num_examples_seen = 0
        for epoch in range(nepoch):
            # Optionally evaluate the loss
            if (epoch % evaluate_loss_after == 0):
                loss = self.calculate_loss(X_train, y_train)
                losses.append((num_examples_seen, loss))
                time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print ("%s: Loss after  epoch=%d: %f" % (time, epoch, loss))
                # Adjust the learning rate if loss increases
                if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                    learning_rate = learning_rate * 0.5 
                    print ("Setting learning rate to %f" % learning_rate)
                    print()
                sys.stdout.flush()
            # For each training example...
            for i in range(len(y_train)):
                # One SGD step
                self.sgd_step(X_train[i], y_train[i], learning_rate)
                num_examples_seen += 1


    
        
    def generate_sentence(self,word):
        x=1
        # We start the sentence with the given start word
        new_sentence = [voc.index(word)]
        sent=word
        while (not new_sentence[-1] == voc.index("sent_end") and (not x>7)):
            x+=1;
            oo,ss=self.forward_propagation(new_sentence)
            l=len(ss)
            #print("seee",l)
            ct=np.random.uniform(0,0,l)
            
            for i in range(l):
                for j in range(len(ss[i])):
                    ct[i]+=self.h[i]*ss[i][j]

            #print("cccc")
            #print(ct)
            n_w=np.array(ct)
            s_w=np.argmax(n_w)
            #sent=sent.join(" ")
            if(not (voc[s_w]=="sent_end")):
                if(not voc[s_w]=="sent_start"  ):
                    sent=sent+" " +(voc[s_w])
            new_sentence.append(s_w) 
            #next_word_probs = self.forward_propagation(new_sentence)
            #sent_len+=1;

            
        # sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]
        print("Generated text")
        print(sent)
        print()


In [51]:
our_rnn=RNNNumpy(len(voc))

In [52]:
print()
print("Training our model with X_tarin and Y_Train")
our_rnn.train_with_sgd(X_train,Y_train)
print()



Training our model with X_tarin and Y_Train
2018-11-23 12:32:19: Loss after  epoch=0: 5.309097


C:\Users\Riya Bhardwaj\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:48: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


2018-11-23 12:32:20: Loss after  epoch=5: 5.287195
2018-11-23 12:32:20: Loss after  epoch=10: 5.264466
2018-11-23 12:32:21: Loss after  epoch=15: 5.239795
2018-11-23 12:32:22: Loss after  epoch=20: 5.211632
2018-11-23 12:32:22: Loss after  epoch=25: 5.177349
2018-11-23 12:32:23: Loss after  epoch=30: 5.130940
2018-11-23 12:32:24: Loss after  epoch=35: 5.028362
2018-11-23 12:32:24: Loss after  epoch=40: 4.666059
2018-11-23 12:32:25: Loss after  epoch=45: 4.399147
2018-11-23 12:32:25: Loss after  epoch=50: 4.306777
2018-11-23 12:32:26: Loss after  epoch=55: 3.963999
2018-11-23 12:32:26: Loss after  epoch=60: 3.844543
2018-11-23 12:32:27: Loss after  epoch=65: 3.265305
2018-11-23 12:32:28: Loss after  epoch=70: 3.159116
2018-11-23 12:32:28: Loss after  epoch=75: 3.086420
2018-11-23 12:32:29: Loss after  epoch=80: 2.619322
2018-11-23 12:32:29: Loss after  epoch=85: 2.204422
2018-11-23 12:32:30: Loss after  epoch=90: 1.413546
2018-11-23 12:32:30: Loss after  epoch=95: 1.069032
2018-11-23 12

In [6]:
start=input("Enter word to start sentence from : ")
our_rnn.generate_sentence(start)

Enter word to start sentence from : the
Generated text
the river All that that that



In [19]:
start=input("Enter word to start sentence from : ")
our_rnn.generate_sentence(start)

Enter word to start sentence from : The
Generated text
The river All All glitters is is gold gold gold



In [42]:
start=input("Enter word to start sentence from : ")
our_rnn.generate_sentence(start)

Enter word to start sentence from : All
Generated text
All river glitters glitters glitters



In [53]:
start=input("Enter word to start sentence from : ")
our_rnn.generate_sentence(start)

Enter word to start sentence from : Today
Generated text
Today river All that glitters glitters is is

